In [1]:
import pandas as pd
import os

In [ ]:
ROOT = '/media/matt/data21/datasets/'
MIMIC_CSV_PATH = os.path.join(ROOT, 'mimic-cxr','data','id_to_findings.csv')
id_findings = pd.read_csv(MIMIC_CSV_PATH)
# reports_clean = pd.read_csv(os.path.join(ROOT, 'mimic-cxr','data','reports_clean.csv'))
labels = pd.read_csv(os.path.join(ROOT, 'mimic-cxr','data','labels', 'mimic-cxr-2.0.0-chexpert.csv.gz'))

In [ ]:
labels.fillna(0, inplace=True)
labels.iloc[:,2:]=labels[labels.columns[2:]].astype(int)

id_findings = id_findings[id_findings.ViewPosition=='AP']

id_findings_labels = id_findings.merge(labels, on='study_id', how='left')
id_findings_labels.drop(columns=['Unnamed: 0', 'subject_id_y'], inplace=True)
id_findings_labels.rename(columns={'subject_id_x':'subject_id'}, inplace=True)
id_findings_labels.reset_index(inplace=True)
id_findings_labels.drop(columns=['index'], inplace=True)
# len(id_findings[id_findings.ViewPosition=='AP'])
## ~60,000

In [ ]:
# ## Assess strat combos:
# from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
# from collections import Counter
# counts = Counter(combination for row in get_combination_wise_output_matrix(y.values, order=4) for combination in row)
# counts.most_common()[::-1]
# # Order 4 yields combinations with 1 sample, this is enough?

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification

x, y = id_findings_labels.iloc[:,:7].copy(), id_findings_labels.iloc[:,7:].copy()
stratifier = IterativeStratification(n_splits=10, order=4)
train_idx, test_idx = next(stratifier.split(x, y))

In [ ]:
pt_set = id_findings_labels.iloc[train_idx]['dicom_id'].reset_index()
pt_set.loc[:,'split']='PT'
ft_set = id_findings_labels.iloc[test_idx]['dicom_id'].reset_index()
ft_set.loc[:,'split']='FT'
split_df = pd.concat([pt_set, ft_set], axis=0).drop(columns='index')
split_df.to_csv(os.path.join(ROOT, 'mimic-cxr','data','id_to_split.csv'), index=False)